In [1]:
import numpy as np
def Fitting_Back_track(v,w,gap,match,mismatch):
    n = len(v)
    m = len(w)
    s = np.zeros((n+1,m+1))
    for i in range(1,len(s[0])):
        s[0][i] = s[0][i-1] - gap

    Backtrack = [[0 for _ in range(m+1)] for _ in range(n+1)]
    for i in range(1,n+1):
        for j in range(1,m+1):
            if v[i-1] == w[j-1]:
                score = match
            else:
                score = -mismatch
            options = [s[i-1][j] - gap,s[i][j-1] - gap,s[i-1][j-1]+score]
            s[i][j] = max(options)
            if s[i][j] == s[i-1][j] - gap:
                Backtrack[i][j] = '↓'
            elif s[i][j] == s[i][j-1] - gap:
                Backtrack[i][j] = '→'
            elif s[i][j] == (s[i-1][j-1]+ score):
                Backtrack[i][j] = '↘'
    return Backtrack,s  
def Output_Fitting(backtrack,s, v, w, i, j, a1, a2):
    if s[i][j] == 0:
        return a1[::-1], a2[::-1]  
    
    if backtrack[i][j] == "↓":
        return Output_Fitting(backtrack,s, v, w, i - 1, j, a1 + v[i-1], a2 + '-')
    elif backtrack[i][j] == "→":
        return Output_Fitting(backtrack,s, v, w, i, j - 1, a1 + '-', a2 + w[j-1])
    else:
        return Output_Fitting(backtrack,s, v, w, i - 1, j - 1, a1 + v[i-1], a2 + w[j-1])
    

def find_max_fitting(backtrack):
    maximum = 0
    max_i = 0
    j = len(backtrack[0])-1
    for i in range(len(backtrack)):
        if backtrack[i][j] > maximum:
            maximum = backtrack[i][j]
            max_i = i
    return maximum,max_i,j
# find_max_fitting(Fitting_Back_track(a,b)[1])

def FittingAlignment(s: str, t: str,
                     match,mismatch, indel_penalty) -> [int, str, str]:
    b,score_mat = Fitting_Back_track(s,t,indel_penalty,match,mismatch)
    score,max_i,max_j = find_max_fitting(score_mat)
    a1,a2 = Output_Fitting(b,score_mat,s,t,max_i,max_j,'','')
    st = ''.join([l for l in a1 if l != '-'])
    for i in range(len(s)-len(st)+1):
        if s[i:i+len(st)] == st:
            index = i+1
            break
    return (a1,a2,index)

In [26]:
template = 'TAAAGTAG'
reads = ["TAAGCA","AGTCAG","GTCAG","TAATAG"]
aligned_reads = []
for read in reads:
    aligned_reads.append(FittingAlignment(template,read,2,1000,1))
aligned_reads

[('TAAAG-TA', 'TAA-GC-A', 1),
 ('AGT-AG', 'AGTCAG', 4),
 ('GT-AG', 'GTCAG', 5),
 ('TAAAGTAG', 'TAA--TAG', 1)]

In [27]:
aligned_reads[0] = ('TAAAGT-A','TAA-G-CA',1)

In [36]:
'''
Algorithm FalconSense
Require: A set of reads R that are aligned on the template S
Ensure: A corrected sequence for S based on the consensus of the reads in
R
1: for every read R ∈ R do
2: Compute the alignment A between R and S allowing matches or indels
(no mismatches);
3: for each aligned position in A do
4: If it is a match aligning S[j] with R[i], add tuple (j, 0, R[i]);
5: If it is a deletion aligning S[j] with −, add tuple (j, 0, −);
6: If it is an insertion aligning − with R[i], suppose this is the dth
insertion after S[j]; add tuple (j, d, R[i]);
7: end for
8: end for

9: For each distinct tuple (p, d, b), let count(p, d, b) be the number of oc-
currences of the tuple in the list.

10: Sort all distinct tuples (p, d, b) by increasing order of p, d and followed
by the alphabetic order of b;
11: for each distinct tuple (p, d, b) in sorted order do
12: if count(p, d, b) > 1

2 x∈{A,C,G,T,−} count(p, 0, x) then

13: Output b;
14: else
15: Output −;
16: end if
17: end for
'''
def FalconSense(aligned_reads):
    consensus_dict = {}
    for read in aligned_reads:
        p = read[2] - 1
        d = 0
        for i in range(len(read[0])):
            if read[0][i] != "-":
                p += 1
                d = 0
            else:
                d += 1
            curr_tuple = (p, d, read[1][i])
            if curr_tuple in consensus_dict:
                consensus_dict[curr_tuple] +=1
            else:
                consensus_dict[curr_tuple] = 1
                
    consensus_data = sorted(consensus_dict.items(), key=lambda x: (x[0][0], x[0][1]))

    consensus_dict = {k: v for k, v in consensus_data}
    options = ['A','C','G','T','-']
    consensus_string = ''
    for tup,count in consensus_dict.items():
        if tup[2] != '-':
            counts = sum(consensus_dict[(tup[0],0,x)] for x in options if (tup[0],0,x) in consensus_dict)
            if count > counts/2:
                consensus_string += tup[2]
            else:
                consensus_string += '-'
    return consensus_dict,consensus_string

In [37]:
FalconSense(aligned_reads)

({(1, 0, 'T'): 2,
  (2, 0, 'A'): 2,
  (3, 0, 'A'): 2,
  (4, 0, '-'): 2,
  (4, 0, 'A'): 1,
  (5, 0, 'G'): 3,
  (5, 0, '-'): 1,
  (6, 0, '-'): 1,
  (6, 0, 'T'): 3,
  (6, 1, 'C'): 3,
  (7, 0, 'A'): 4,
  (8, 0, 'G'): 3},
 'TAA-GTCAG')